In [1]:
import pandas as pd
import numpy as np
import os

wd = os.getcwd()

In [2]:
n_agents = [20, 40, 60, 80]
instances = set(np.arange(1,101))
dissappear_at_target = ["DaT", "noDaT"]
waiting = ["wW", "noW"]
suboptimality = ["1", "1.1"]
results = {}
time_limit = 60
for d in dissappear_at_target:
    for w in waiting:
        for s in suboptimality:
            
            map_name = "empty-16-16"
            setting_name = f"{d}-{w}-suboptimality_{s}"
            file_name = f"/results-{s}/{setting_name}/{map_name}-{setting_name}.csv"
            df = pd.read_csv(wd+file_name)
            str1 = "./instances/empty-16-16/empty-16-16-random-"
            str2 = "instances/empty-16-16/empty-16-16-random-"
            str3 = str1 if df.loc[0, 'instance name'].startswith("./") else str2
            df['instance name'] = df['instance name'].str.replace(str3, "", regex=True)
            series = df['instance name'].str.split("-")
            df['n_agents'] = series.apply(lambda arr: int(arr[0]))
            df['instance_no'] = series.apply(lambda arr: int(arr[-1].split(".")[0]))
            
            #print(d, w, s, df.shape[0])
            for k in n_agents:
                instances_ = set(df.loc[df['n_agents'] == k, 'instance_no'].values)
                missing = instances.difference(instances_)
                for instance_no in missing:
              
                    df.loc[len(df)] = -1
                    df.loc[len(df)-1, ['runtime', 'instance_no']] = time_limit, instance_no
            
            df.sort_values(['n_agents', 'instance_no'], inplace=True)
            df['map'] = 'empty-16-16'
            df['disappear_at_target'] = d
            df['wait-action'] = w
            results[d + w + s] = df        

In [7]:
lst = []
for d in dissappear_at_target:
    for w in waiting:
        for s in suboptimality:
            if s == '1':
                df_ = results[d + w + s]
                for k in n_agents:
                    df = df_.loc[df_['n_agents'] == k]
                    sol_rate = 1 - df[df['solution cost'] == -1].shape[0]/df.shape[0]

                    print(d, w, "subopt:",s, k, "agents | solution rate:", round(sol_rate,3))
                    
                    lst.append([d, w, k, round(sol_rate, 2)])

DaT wW subopt: 1 20 agents | solution rate: 1.0
DaT wW subopt: 1 40 agents | solution rate: 1.0
DaT wW subopt: 1 60 agents | solution rate: 0.62
DaT wW subopt: 1 80 agents | solution rate: 0.16
DaT noW subopt: 1 20 agents | solution rate: 1.0
DaT noW subopt: 1 40 agents | solution rate: 0.99
DaT noW subopt: 1 60 agents | solution rate: 0.66
DaT noW subopt: 1 80 agents | solution rate: 0.21
noDaT wW subopt: 1 20 agents | solution rate: 1.0
noDaT wW subopt: 1 40 agents | solution rate: 0.91
noDaT wW subopt: 1 60 agents | solution rate: 0.08
noDaT wW subopt: 1 80 agents | solution rate: 0.0
noDaT noW subopt: 1 20 agents | solution rate: 1.0
noDaT noW subopt: 1 40 agents | solution rate: 0.939
noDaT noW subopt: 1 60 agents | solution rate: 0.149
noDaT noW subopt: 1 80 agents | solution rate: 0.0


In [4]:
merged_df = pd.DataFrame(columns=["map", "n_agents", "instance_no", "disappear_at_target", "wait-action", "opt_runtime", "subopt_runtime", "opt_cost", "subopt_cost"])
for d in dissappear_at_target:
    for w in waiting:
        df1 = results[d + w + "1"]
        df2 = results[d + w + "1.1"]
        df3 = pd.DataFrame(columns=["opt_cost", "subopt_cost", "opt_runtime", "subopt_runtime"])
        df3[["opt_cost", "opt_runtime"]] = df1[['solution cost', 'runtime']]
        df3[["subopt_cost", "subopt_runtime"]] = df2[['solution cost', 'runtime']]
        df3[["map", "n_agents", "instance_no", "disappear_at_target", "wait-action"]] = df1[["map", "n_agents", "instance_no", "disappear_at_target", "wait-action"]]
        merged_df = pd.concat((merged_df, df3))

In [5]:
merged_df

,map,n_agents,instance_no,disappear_at_target,wait-action,opt_runtime,subopt_runtime,opt_cost,subopt_cost
0,empty-16-16,20,1,DaT,wW,0.006,0.004,204,204
1,empty-16-16,20,2,DaT,wW,0.001,0.001,260,260
2,empty-16-16,20,3,DaT,wW,0.001,0.001,204,204
3,empty-16-16,20,4,DaT,wW,0.001,0.001,244,244
4,empty-16-16,20,5,DaT,wW,0.001,0.000,226,226
...,...,...,...,...,...,...,...,...,...
368,empty-16-16,80,96,noDaT,noW,60.001,60.001,-1,-1
369,empty-16-16,80,97,noDaT,noW,60.001,0.421,-1,1005
370,empty-16-16,80,98,noDaT,noW,60.001,0.365,-1,955
371,empty-16-16,80,99,noDaT,noW,60.001,0.449,-1,988


In [6]:
merged_df.to_excel("eecbs_opt_subopt_results.xlsx")